In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils

In [2]:
#set random seed for purposes of reproducibility
seed = 21

Preparation of data

In [3]:
from keras.datasets import cifar10

In [4]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [5]:
#normalize the inpust from 0-255 to between 0 and 1 by dividing by 255
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [6]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
class_num = y_test.shape[1]

Creating model

In [7]:
model = Sequential()

In [8]:
model.add(Conv2D(32, (3, 3), input_shape = X_train.shape[1:], padding = 'same'))
model.add(Activation('relu'))

In [9]:
model.add(Dropout(0.2)) #to avoid overfitting

In [10]:
model.add(BatchNormalization())

In [11]:
model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(Activation('relu'))

In [12]:
#объединяющий слой
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [13]:
model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), padding = 'same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [14]:
model.add(Flatten())
model.add(Dropout(0.2))

In [15]:
model.add(Dense(256, kernel_constraint = maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(128, kernel_constraint = maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [16]:
model.add(Dense(class_num))
model.add(Activation('softmax'))

In [17]:
epochs = 25
optimizer = 'adam'

In [18]:
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])

In [19]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0

Training

In [20]:
np.random.seed(seed)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = epochs, batch_size = 64)

Epoch 1/25
782/782 [==============================] - 248s 318ms/step - loss: 1.5111 - accuracy: 0.4656 - val_loss: 1.3572 - val_accuracy: 0.5200
Epoch 2/25
782/782 [==============================] - 223s 285ms/step - loss: 1.0308 - accuracy: 0.6345 - val_loss: 0.8627 - val_accuracy: 0.6943
Epoch 3/25
782/782 [==============================] - 226s 289ms/step - loss: 0.8591 - accuracy: 0.6967 - val_loss: 0.8636 - val_accuracy: 0.6996
Epoch 4/25
782/782 [==============================] - 224s 287ms/step - loss: 0.7645 - accuracy: 0.7318 - val_loss: 0.6916 - val_accuracy: 0.7629
Epoch 5/25
782/782 [==============================] - 226s 289ms/step - loss: 0.7075 - accuracy: 0.7528 - val_loss: 0.6882 - val_accuracy: 0.7606
Epoch 6/25
782/782 [==============================] - 236s 302ms/step - loss: 0.6618 - accuracy: 0.7685 - val_loss: 0.6590 - val_accuracy: 0.7731
Epoch 7/25
782/782 [==============================] - 236s 302ms/step - loss: 0.6281 - accuracy: 0.7823 - val_loss: 0.6450 -

Model evaluation

In [21]:
scores = model.evaluate(X_test, y_test, verbose = 0)
print('Accuracy: %.2f%%' % (scores[1] * 100))

Accuracy: 81.78%
